### Question 1.
#### Author: Lin, Zilong (zillin) — Gopalkrishna, Sumanth (sgopalk)

#### From the training set, compute the total number of unique words in the set and the count of each unique word in each message. Hence, if there are N unique words and M messages in the training set, then the count of each unique word for all messages should result in a M × N matrix. You may want to use DataFrame and dictionary objects to accomplish this. You may also use split() to ignore whitespace.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix


def loadData(filename):
    df = pd.read_csv(filename)
    X_content = df.loc[:, "Message"]
    X_split = X_content.copy()
    for i in X_content.index:
        X_line = X_content.loc[i]
        words = [x.strip() for x in X_line.strip().split(" ") if x != ""]
        X_split.loc[i] = words
    y = df.loc[:, "Label"].to_numpy()
    return X_split, y


def uniqueWords(X_split):
    all_words = []
    for i in X_split.index:
        for w in X_split.loc[i]:
            if w not in all_words:
                all_words.append(w)
    return all_words


def sent2matrix(X_split, all_words):
    X = np.zeros((len(list(X_split.index)), len(all_words)))
    for order, i in enumerate(X_split.index):
        for w in X_split.loc[i]:
            if w in all_words:
                X[order, all_words.index(w)] += 1
    return X

X_split, y = loadData(filename="message.csv")
skfolds = StratifiedShuffleSplit(n_splits=10, random_state=42, test_size=0.2)

count = 0
for train_index, test_index in skfolds.split(X_split, y):
  count += 1
  print("The {}th fold".format(count))
  X_train_split = X_split.loc[train_index]
  y_train = y[train_index]
  X_test_split = X_split.loc[test_index]
  y_test = y[test_index]

  all_words = uniqueWords(X_split)
  X_train = sent2matrix(X_train_split, all_words)
  X_test = sent2matrix(X_test_split, all_words)
  print("Total unique word number is {}".format(len(all_words)))
  print("The M × N matrix of training set is {}".format(X_train))

The 1th fold
Total unique word number is 8753
The M × N matrix of training set is [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
The 2th fold
Total unique word number is 8753
The M × N matrix of training set is [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
The 3th fold
Total unique word number is 8753
The M × N matrix of training set is [[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
The 4th fold
Total unique word number is 8753
The M × N matrix of training set is [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
The 5th fold
Total unique word number is 8753
Th

#### Perform maximum likelihood estimation to determine the prior and class conditional probabilities of the training set (e.g. compute P(y = 1), P(y = 0), P(xi|y = 0), and P(xi|y = 1)) ,where xi represents the i-th unique word. Be sure to confirm that these are indeed probabilities.

In [ ]:
def MLE(X, y):
  # calculate P(y = 1), P(y = 0)
  p_y_1 = np.sum(y) / y.shape[0]
  p_y_0 = 1 - np.sum(y) / y.shape[0]

  # calculate P(xi|y = 0), and P(xi|y = 1)) ,where xi represents the i-th unique word
  p_z_y_0 = np.zeros((X.shape[-1]))
  p_z_y_1 = np.zeros((X.shape[-1]))
  # use Laplace smoothing to calculate P(xi|y = 0), and P(xi|y = 1))
  laplace_p_z_y_0 = np.zeros((X.shape[-1]))
  laplace_p_z_y_1 = np.zeros((X.shape[-1]))
  # calculate the conditional probability without Laplace smoothing and with Laplace smoothing
  for i in range(X.shape[-1]):
      z_sum = np.sum(X[:, i])  # The sum of variable z_i
      z_sum_y_1 = np.sum(X[:, i] * y) # The sum of variable z_i when y = 1
      z_sum_y_0 = z_sum - z_sum_y_1 # The sum of variable z_i when y = 0
      p_z_y_0[i] = z_sum_y_0 / z_sum
      p_z_y_1[i] = z_sum_y_1 / z_sum
      laplace_p_z_y_0[i] = (z_sum_y_0 + 1) / (z_sum + 2)
      laplace_p_z_y_1[i] = (z_sum_y_1 + 1) / (z_sum + 2)
  return p_y_0, p_y_1, p_z_y_0, p_z_y_1, laplace_p_z_y_0, laplace_p_z_y_1

In [ ]:
X_split, y = loadData(filename="message.csv")
# skfolds = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
skfolds = StratifiedShuffleSplit(n_splits=10, random_state=42, test_size=0.2)

acc_y_test = []
acc_y_pred = []
count = 0
for train_index, test_index in skfolds.split(X_split, y):
  count += 1
  print("The {}th fold".format(count))
  X_train_split = X_split.loc[train_index]
  y_train = y[train_index]
  X_test_split = X_split.loc[test_index]
  y_test = y[test_index]
  # print("shape: {}, {}".format(y_train.shape, y_test.shape))

  all_words = uniqueWords(X_split)
  X_train = sent2matrix(X_train_split, all_words)
  X_test = sent2matrix(X_test_split, all_words)

  p_y_0, p_y_1, p_z_y_0, p_z_y_1, laplace_p_z_y_0, laplace_p_z_y_1 = MLE(X_train, y_train)
  
  print("P(y = 0) = {}".format(p_y_0))
  print("P(y = 1) = {}".format(p_y_1))
  print("Without Laplace smoothing P(xi|y = 0) = {}".format(p_z_y_0))
  print("Without Laplace smoothing P(xi|y = 1) = {}".format(p_z_y_1))
  print("After Laplace smoothing P(xi|y = 0) = {}".format(laplace_p_z_y_0))
  print("After Laplace smoothing P(xi|y = 1) = {}".format(laplace_p_z_y_1))

The 1th fold


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars


P(y = 0) = 0.8658290329818263
P(y = 1) = 0.13417096701817366
Without Laplace smoothing P(xi|y = 0) = [0.87892377 0.81818182 1.         ... 1.         1.         1.        ]
Without Laplace smoothing P(xi|y = 1) = [0.12107623 0.18181818 0.         ... 0.         0.         0.        ]
After Laplace smoothing P(xi|y = 0) = [0.87555556 0.79166667 0.66666667 ... 0.66666667 0.66666667 0.66666667]
After Laplace smoothing P(xi|y = 1) = [0.12444444 0.20833333 0.33333333 ... 0.33333333 0.33333333 0.33333333]
The 2th fold
P(y = 0) = 0.8658290329818263
P(y = 1) = 0.13417096701817366
Without Laplace smoothing P(xi|y = 0) = [0.89237668 0.8        1.         ... 1.         1.         1.        ]
Without Laplace smoothing P(xi|y = 1) = [0.10762332 0.2        0.         ... 0.         0.         0.        ]
After Laplace smoothing P(xi|y = 0) = [0.88888889 0.77272727 0.66666667 ... 0.66666667 0.66666667 0.66666667]
After Laplace smoothing P(xi|y = 1) = [0.11111111 0.22727273 0.33333333 ... 0.33333333 

#### Once the above probabilities are determined, use Naive Bayes classification to classify each of the testing examples as spam or not. Ignore words from the testing set that are not contained in the training set. Report the accuracy, precision, recall and specificity, along with the confusion matrix for each fold. Also report the average accuracy, precision, recall and specificity over all folds.

In [ ]:
def NB(p_y_0, p_y_1, p_z_y_0, p_z_y_1, X_test):
    y_pred = np.zeros((X_test.shape[0],))
    for i in range(X_test.shape[0]):
        line = X_test[i]
        p_0 = 1 * p_y_0
        p_1 = 1 * p_y_1
        for j in range(line.shape[0]):
            p_0 *= pow(p_z_y_0[j], line[j])
            p_1 *= pow(p_z_y_1[j], line[j])
        if p_0 > p_1:
            y_pred[i] = 0
        elif p_0 < p_1:
            y_pred[i] = 1
        else:
            y_pred[i] = 0
            # print("balance")
    return y_pred


def _metrics(y_test, y_pred):
    accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
    precision = precision_score(y_true=y_test, y_pred=y_pred)
    recall = recall_score(y_true=y_test, y_pred=y_pred)
    confusion_mat = confusion_matrix(y_true=y_test, y_pred=y_pred)
    return accuracy, precision, recall, confusion_mat

In [ ]:
print("Running Naive Bayes without Laplace Smoothing")
X_split, y = loadData(filename="message.csv")
# skfolds = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
skfolds = StratifiedShuffleSplit(n_splits=10, random_state=42, test_size=0.2)

acc_y_test = []
acc_y_pred = []
count = 0
for train_index, test_index in skfolds.split(X_split, y):
  count += 1
  print("The {}th fold".format(count))
  X_train_split = X_split.loc[train_index]
  y_train = y[train_index]
  X_test_split = X_split.loc[test_index]
  y_test = y[test_index]
  # print("shape: {}, {}".format(y_train.shape, y_test.shape))

  all_words = uniqueWords(X_split)
  X_train = sent2matrix(X_train_split, all_words)
  X_test = sent2matrix(X_test_split, all_words)

  p_y_0, p_y_1, p_z_y_0, p_z_y_1, _, _ = MLE(X_train, y_train)
  y_test_pred = NB(p_y_0, p_y_1, p_z_y_0, p_z_y_1, X_test)
  accuracy, precision, recall, confusion_mat = _metrics(y_test, y_test_pred)
  print("Accuracy: {}, Precision: {}, Recall: {}, Confusion matrix: {}".format(accuracy, precision, recall, confusion_mat))

  acc_y_test = np.concatenate((acc_y_test, y_test), axis=0)
  acc_y_pred = np.concatenate((acc_y_pred, y_test_pred), axis=0)

#  average accuracy, precision, recall
accuracy, precision, recall, _ = _metrics(acc_y_test, acc_y_pred)
print("Average accuracy, precision, recall and specificity over all folds \nAccuracy: {}, Precision: {}, Recall: {}".format(accuracy, precision, recall))

Running Naive Bayes without Laplace Smoothing
The 1th fold


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars


Accuracy: 0.9049327354260089, Precision: 0.9777777777777777, Recall: 0.2953020134228188, Confusion matrix: [[965   1]
 [105  44]]
The 2th fold
Accuracy: 0.9165919282511211, Precision: 0.9827586206896551, Recall: 0.3825503355704698, Confusion matrix: [[965   1]
 [ 92  57]]
The 3th fold
Accuracy: 0.9121076233183857, Precision: 0.9811320754716981, Recall: 0.348993288590604, Confusion matrix: [[965   1]
 [ 97  52]]
The 4th fold
Accuracy: 0.9183856502242153, Precision: 0.967741935483871, Recall: 0.40268456375838924, Confusion matrix: [[964   2]
 [ 89  60]]
The 5th fold
Accuracy: 0.915695067264574, Precision: 0.9661016949152542, Recall: 0.3825503355704698, Confusion matrix: [[964   2]
 [ 92  57]]
The 6th fold
Accuracy: 0.9112107623318386, Precision: 0.9807692307692307, Recall: 0.3422818791946309, Confusion matrix: [[965   1]
 [ 98  51]]
The 7th fold
Accuracy: 0.9130044843049328, Precision: 0.9814814814814815, Recall: 0.35570469798657717, Confusion matrix: [[965   1]
 [ 96  53]]
The 8th fold


In [ ]:
print("Running Naive Bayes with Laplace Smoothing")
X_split, y = loadData(filename="message.csv")
skfolds = StratifiedShuffleSplit(n_splits=10, random_state=42, test_size=0.2)

acc_y_test = []
acc_y_pred = []
count = 0
for train_index, test_index in skfolds.split(X_split, y):
  count += 1
  print("The {}th fold".format(count))
  X_train_split = X_split.loc[train_index]
  y_train = y[train_index]
  X_test_split = X_split.loc[test_index]
  y_test = y[test_index]

  all_words = uniqueWords(X_split)
  X_train = sent2matrix(X_train_split, all_words)
  X_test = sent2matrix(X_test_split, all_words)

  p_y_0, p_y_1, _, _, laplace_p_z_y_0, laplace_p_z_y_1 = MLE(X_train, y_train)
  y_test_pred = NB(p_y_0, p_y_1, laplace_p_z_y_0, laplace_p_z_y_1, X_test)
  accuracy, precision, recall, confusion_mat = _metrics(y_test, y_test_pred)
  print("Accuracy: {}, Precision: {}, Recall: {}, Confusion matrix: {}".format(accuracy, precision, recall, confusion_mat))

  acc_y_test = np.concatenate((acc_y_test, y_test), axis=0)
  acc_y_pred = np.concatenate((acc_y_pred, y_test_pred), axis=0)

#  average accuracy, precision, recall
accuracy, precision, recall, _ = _metrics(acc_y_test, acc_y_pred)
print("Average accuracy, precision, recall and specificity over all folds \nAccuracy: {}, Precision: {}, Recall: {}".format(accuracy, precision, recall))

Running Naive Bayes with Laplace Smoothing
The 1th fold


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars


Accuracy: 0.947085201793722, Precision: 1.0, Recall: 0.6040268456375839, Confusion matrix: [[966   0]
 [ 59  90]]
The 2th fold
Accuracy: 0.9390134529147982, Precision: 1.0, Recall: 0.5436241610738255, Confusion matrix: [[966   0]
 [ 68  81]]
The 3th fold
Accuracy: 0.9426008968609866, Precision: 1.0, Recall: 0.5704697986577181, Confusion matrix: [[966   0]
 [ 64  85]]
The 4th fold
Accuracy: 0.9551569506726457, Precision: 1.0, Recall: 0.6644295302013423, Confusion matrix: [[966   0]
 [ 50  99]]
The 5th fold
Accuracy: 0.9452914798206278, Precision: 1.0, Recall: 0.5906040268456376, Confusion matrix: [[966   0]
 [ 61  88]]
The 6th fold
Accuracy: 0.9408071748878923, Precision: 1.0, Recall: 0.5570469798657718, Confusion matrix: [[966   0]
 [ 66  83]]
The 7th fold
Accuracy: 0.9417040358744395, Precision: 1.0, Recall: 0.5637583892617449, Confusion matrix: [[966   0]
 [ 65  84]]
The 8th fold
Accuracy: 0.9417040358744395, Precision: 1.0, Recall: 0.5637583892617449, Confusion matrix: [[966   0]
 [

#### Write a paragraph the summarizes the results and your thoughts about Naive Bayes classification for this problem

Based on the Maximum likelihood Estimation, we can find that P(y = 0) = 0.87, P(y = 1) = 0.13. This means that the dataset is unbalanced. This makes the prediction tends to Label 0, which is shown in the result. In the result, Precision is very high, while Recall is not good enough. This is also reflected in the confusion matrix, in which the more data of Label 1 were predicted as Label 0, while no data of Label 0 were predicted as Label 1.

### Discuss how the results after Laplace Smoothing differ from the prior results.

Based on the result, after applying Laplace smoothing, accuracy, precision and recall rises, in which recall rise much higher. Given that the data of y=1 is much samller than that of y=0, the smoothing help the model release the bias in the result, as false positive decreases remarkably.